In [21]:
import numpy as np
import pandas as pd
import datetime

mydateparser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
CGM_DATA = pd.read_csv('CGMData.csv', parse_dates=['Date'], date_parser=mydateparser)
CGM_DATA = CGM_DATA[["Date", "Time", "Sensor Glucose (mg/dL)"]]
CGM_DATA.rename({'Sensor Glucose (mg/dL)': 'Glucose'}, axis=1, inplace=True)
#print(CGM_DATA)
CGM_DATA['Time'] = pd.to_timedelta(CGM_DATA['Time'])
#print(CGM_DATA)


'''
1)         Hyperglycemia: CGM > 180 mg/dL
2)Hyperglycemia Critical: CGM > 250 mg/dL
3)          Normal Range: CGM <= 180 mg/dL && CGM >= 70 mg/dL
4)       Secondary Range: CGM >= 70 mg/dL  && CGM <= 150 mg/dL
5)  Hypoglycemia Level 1: CGM < 70 mg/dL
6)  Hypoglycemia Level 2: CGM < 54 mg/dL
'''
def a (level):
    if level > 250:
        return 1
    elif level > 180 and level <= 250:
        return 2
    elif level > 150 and level <= 180:
        return 3
    elif level >= 70 and level <= 150:
        return 4
    elif level >= 54 and level < 70:
        return 5
    elif level < 54:
        return 6
    else:
        return 0

CGM_levels = pd.DataFrame()
CGM_levels['CGM_Category'] = CGM_DATA.apply(lambda row: a(row['Glucose']), axis=1)
#print(CGM_levels)

Final_CGM_Data = CGM_DATA.join(CGM_levels)
Final_CGM_Data = Final_CGM_Data.reset_index(drop=True)
#print(Final_CGM_Data)


'''
1)  Day_time = 6AM to 12AM
2) OverNight = 12AM to 6AM
3) Whole_Day = 12AM to 12AM
'''
Midnight = datetime.timedelta(hours=0, minutes=0, seconds=0)
EarlyDay = datetime.timedelta(hours=6, minutes=0, seconds=0)
BeforeEarlyDay = datetime.timedelta(hours=5, minutes=59, seconds=59)
EndOfTheDay = datetime.timedelta(hours=23, minutes=59, seconds=59)

Time_Interval = pd.DataFrame()

def b (line):
    if line >= Midnight and line <= BeforeEarlyDay:
        return 'overnight'
    elif line >= EarlyDay and line <= EndOfTheDay:
        return 'daytime'
    else:
        return 0


Time_Interval['Time_Category'] = Final_CGM_Data.apply(lambda row: b(row['Time']), axis=1)
#print(Time_Interval)

Final_Time_Data = Final_CGM_Data.join(Time_Interval)
Final_Time_Data = Final_Time_Data.reset_index(drop=True)
#print(Final_Time_Data)


mydateparser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
Insulin = pd.read_csv('InsulinData.csv', parse_dates=['Date'], date_parser=mydateparser)
Insulin = Insulin[["Date", "Time", "Alarm"]]
Insulin['Time'] = pd.to_timedelta(Insulin['Time'])
Insulin['Alarm'] = Insulin['Alarm'].astype(str)
#print(Insulin)

#print(sorted((Insulin.loc[Insulin['Alarm'] == 'AUTO MODE ACTIVE PLGM OFF']),asc)
NewTable = pd.DataFrame((Insulin.loc[Insulin['Alarm'] == 'AUTO MODE ACTIVE PLGM OFF']))
NewTable = NewTable.sort_values(by='Date', ascending= 1)
NewTable = NewTable.reset_index(drop=True)
#print(NewTable)
start_date = NewTable['Date'].iloc[0]
start_time = NewTable['Time'].iloc[0]
#print(start_date)
#print(start_time)


Insulin_Mode = pd.DataFrame()
#d=date t=time
def c (d, t):
    if d < start_date:
        return 'Manual_Mode'
    elif d == start_date:
        if t < start_time:
            return 'Manual_Mode'
        else:
            return 'Auto_Mode'
    else:
        return 'Auto_Mode'

Insulin_Mode['Mode'] = Final_Time_Data.apply(lambda row: c(row['Date'], row['Time']), axis=1)
#print(Insulin_Mode)

Final_Insulin_Data = Final_Time_Data.join(Insulin_Mode)
Final_Insulin_Data = Final_Insulin_Data.reset_index(drop=True)
#print(Final_Insulin_Data)

def d (level):
     if level == 1 or level == 2:
         return 12
     elif level == 3 or level == 4:
         return 34
     elif level == 5 or level == 6:
         return 56
     else:
         return 0


Combined_Data = pd.DataFrame()
Combined_Data['Overlay'] = Final_Insulin_Data.apply(lambda row: d(row['CGM_Category']), axis=1)
#print(Combined_Data)

Final_Combined_Data = Final_Insulin_Data.join(Combined_Data)
Final_Combined_Data = Final_Combined_Data.reset_index(drop=True)
#print(Final_Combined_Data)

Total_Combined_Data_Glucose = Final_Combined_Data.groupby(['Mode', 'Date', 'Time_Category', 'Overlay']).size().reset_index().rename(columns={0: 'TotalCount'})
Total_Separate_Data_Glucose = Final_Combined_Data.groupby(['Mode', 'Date', 'Time_Category', 'CGM_Category']).size().reset_index().rename(columns={0: 'TotalCount'})
Total_Combined_Data_Time = Final_Combined_Data.groupby(['Mode', 'Date', 'Overlay']).size().reset_index().rename(columns={0: 'TotalCount'})
Total_Separate_Data_Time = Final_Combined_Data.groupby(['Mode', 'Date', 'CGM_Category']).size().reset_index().rename(columns={0: 'TotalCount'})

#print(Total_Combined_Data_Glucose)

Manual_Result = []
Auto_Result = []


#Function to divide each row by 288
def to_divide(dividend):
  if(np.isnan(dividend) == False):
    return dividend/288



def run_query_1(overlay_val,time_val,mode_val):
    if time_val=='whole_day':
        Range = Total_Combined_Data_Glucose.query("(Overlay == @overlay_val) and (Mode == @mode_val)").reset_index()
    else:
        Range = Total_Combined_Data_Glucose.query("(Overlay == @overlay_val) and (Time_Category == @time_val) and (Mode == @mode_val)").reset_index()
    if(Range.empty != True):
        Result_Range = pd.DataFrame()
        Result_Range['DividedCount'] = Range.apply(lambda row: to_divide(row['TotalCount']), axis=1)
        Mean_Range = Result_Range['DividedCount'].mean()
    else:
        Mean_Range = 0
        
    print("for overlay_val: " + str(overlay_val) + " , time_val: "+ time_val + ", mode_val: " + mode_val + " mean is: "+ str(Mean_Range))
    if mode_val == 'Manual_Mode':
        Manual_Result.append(Mean_Range)
    else:
        Auto_Result.append(Mean_Range)

def run_query_2(overlay_val,time_val,mode_val):
    if time_val=='whole_day':
        Range = Total_Separate_Data_Glucose.query("(CGM_Category == @overlay_val) and (Mode == @mode_val)").reset_index()
    else:
        Range = Total_Separate_Data_Glucose.query("(CGM_Category == @overlay_val) and (Time_Category == @time_val) and (Mode == @mode_val)").reset_index()
    if(Range.empty != True):
        Result_Range = pd.DataFrame()
        Result_Range['DividedCount'] = Range.apply(lambda row: to_divide(row['TotalCount']), axis=1)
        Mean_Range = Result_Range['DividedCount'].mean()
    else:
        Mean_Range = 0
        
    print("for overlay_val: " + str(overlay_val) + " , time_val: "+ time_val + ", mode_val: " + mode_val + " mean is: "+ str(Mean_Range))
    if mode_val == 'Manual_Mode':
        Manual_Result.append(Mean_Range)
    else:
        Auto_Result.append(Mean_Range)

total_overlay_values_1 = [12,34,56]
total_overlay_values_2 = [1,4,6]
total_time_vales = ['overnight','daytime','whole_day']
total_mode_vales = ['Manual_Mode','Auto_Mode']

for val_1 in total_overlay_values_1:
    for val_3 in total_mode_vales:
        for val_2 in total_time_vales:
            run_query_1(val_1,val_2,val_3)
            
for val_4 in total_overlay_values_2:
    for val_6 in total_time_vales:
        for val_5 in total_mode_vales:
            run_query_2(val_4,val_6,val_5)

            
print(Manual_Result)
print(Auto_Result)

Result = pd.DataFrame()
Result['Manual'] = Manual_Result
Result['Auto'] = Auto_Result

Result= Result.transpose()
print(Result)
Result.to_csv("NoLastName_Result.csv",index=True,header=False)


for overlay_val: 12 , time_val: overnight, mode_val: Manual_Mode mean is: 0.10367063492063491
for overlay_val: 12 , time_val: daytime, mode_val: Manual_Mode mean is: 0.24351851851851852
for overlay_val: 12 , time_val: whole_day, mode_val: Manual_Mode mean is: 0.1990214646464646
for overlay_val: 12 , time_val: overnight, mode_val: Auto_Mode mean is: 0.05644478319783198
for overlay_val: 12 , time_val: daytime, mode_val: Auto_Mode mean is: 0.19754603620474406
for overlay_val: 12 , time_val: whole_day, mode_val: Auto_Mode mean is: 0.15304487179487167
for overlay_val: 34 , time_val: overnight, mode_val: Manual_Mode mean is: 0.18240740740740738
for overlay_val: 34 , time_val: daytime, mode_val: Manual_Mode mean is: 0.3483072916666667
for overlay_val: 34 , time_val: whole_day, mode_val: Manual_Mode mean is: 0.26803315412186385
for overlay_val: 34 , time_val: overnight, mode_val: Auto_Mode mean is: 0.20265683520599262
for overlay_val: 34 , time_val: daytime, mode_val: Auto_Mode mean is: 0.4511